In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


navigate to Case Research folder

In [6]:
%cd /content/drive/MyDrive/twitterScrape

/content/drive/MyDrive/twitterScrape


Set Tensorflow to  1.15

In [7]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


Install needed libraries here

In [8]:
!apt install -qq enchant
!pip install pyenchant

The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 13 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package libtext-iconv-perl.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../0-libtext-iconv-perl_1.7-5build6_amd64.deb ...
Unpacking libtext-iconv-perl (1.7-5build6) ...
Selecting previously unselected package libaspe

TwitterClassify

In [ ]:
import string
import time
import os
import tensorflow as tf
from tensorflow.contrib import predictor
import numpy as np
import pandas as pd
import os
import enchant
import csv


def _bytes_feature(value):
    bValue = bytes(value, 'utf-8')
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[bValue]))


def runPrediction(msg, predModel):
    nMsg = msg;

    feature = {
        'tweetText': _bytes_feature(nMsg)
    }
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    serialized_example = example.SerializeToString()
    results = predModel({"inputs": [serialized_example]})
    results = results.get('scores')[0]
    print(results)
    results = str(results).strip('[')
    results = str(results).strip(']')
    return results

def main():
    top_emotions = ["robbysoave", "RBReich", "PalmerReport", "AddInfoOrg","Public_Citizen","jimsciutto", "DouglasKMurray","MattWalshBlog", "realdonaldtrump", "tuckercarlson", "JesseKellyDC"]
    top_emotions.sort()
    headCats = ["emotions","tweetText"]
    headCats.extend(top_emotions)
    en_us = enchant.Dict("en_US")
    csvFile = open('twitter_Categories.csv', 'w', newline='')
    csvWriter = csv.writer(csvFile)
    csvWriter.writerow(headCats)
    #load trained model
    predict_fn = predictor.from_saved_model("export/1602022423")

    data = pd.read_csv('reddit_hot_NNN_v7.csv', header=0)
    #np.random.shuffle(data.values[1:, :])
    data.head()
    # tweetText is the text being evaluated
    tweetText = data['tweetText']
    data['emotions'] = data['emotions'].str.strip('[')
    data['emotions'] = data['emotions'].str.strip(']')
    data['emotions'] = data['emotions'].str.replace(" ", "")
    data['emotions'] = data['emotions'].str.split(',')
    emotion = data['emotions']

    emotionTest = data['emotions']
    emotionTest = list(set(emotionsTest))

    
    for i in range(len(tweetText)):

        if isinstance(tweetText[i], str):
            pMsg = runPrediction(tweetText[i], predict_fn)
            sMsg = pMsg.split(" ")
            outMsg = [user[i],tweetText[i]]
            for m in sMsg:
                if len(m) > 0:
                    outMsg.append(m)
            csvWriter.writerow(outMsg)

if __name__ == "__main__":
    main()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from export/1602022423/variables/variables


NotFoundError: ignored

Train the estimator

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import tensorflow_hub as hub
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer



def main():


    data = pd.read_csv('twitter_emotions.csv', header = 0)
    np.random.shuffle(data.values[1:, :])
    data.head()
    descriptions = data['twitter_emotion']
    data['emotion'] = data['emotion'].str.strip('[')
    data['emotion'] = data['emotion'].str.strip(']')
    data['emotion'] = data['emotion'].str.replace(" ", "")
    data['emotion'] = data['emotion'].str.split(',')

    emotion = data['user']



    top_emotions = ["Anger","Anticipation","Disgust","Joy"]
    print(len(top_emotions))
    train_size = int(len(descriptions)* .8)
    train_descriptions = descriptions[:train_size].astype('str')
    train_emotions = emotion[:train_size]

    test_descriptions = descriptions[train_size:].astype('str')
    test_emotions = emotion[train_size:]

    description_embeddings = hub.text_embedding_column(
      "tweetText",
      module_spec="https://tfhub.dev/google/universal-sentence-encoder-large/3"
    )

    encoder = MultiLabelBinarizer()
    encoder.fit_transform(train_emotions)
    train_encoded = encoder.transform(train_emotions)
    test_encoded = encoder.transform(test_emotions)
    num_classes = len(encoder.classes_)
    print(num_classes)
    print(encoder.classes_)
    print(train_encoded[0])

    multi_label_head = tf.contrib.estimator.multi_label_head(
        num_classes,
        loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
    )

    feature_columns = [description_embeddings]
    estimator = tf.estimator.DNNEstimator(
        head=multi_label_head,
        hidden_units=[64,10],
        feature_columns=feature_columns,
        model_dir='models/twitterEmot'
    )

    # Format our data for the numpy_input_fn
    features = {
      'tweetText': np.array(train_descriptions)
    }
    labels = np.array(train_encoded)

    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        features,
        labels,
        shuffle=True,
        batch_size=32,
        num_epochs=200,
    )


    estimator.train(input_fn=train_input_fn)
    feature_spec = tf.feature_column.make_parse_example_spec(feature_columns)
    print(feature_spec)
    # Build receiver function, and export.
    serving_input_receiver_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
    export_dir = estimator.export_savedmodel('export', serving_input_receiver_fn)

    eval_input_fn = tf.estimator.inputs.numpy_input_fn({"tweetText": np.array(test_descriptions).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)

    estimator.evaluate(input_fn=eval_input_fn)
    raw_test = [
        "This stupid machine never works.",
        # Documentary
        "I really like ice cream",
        # Action, Adventure
        "I'm afraid of spiders",
        # Comedy
    ]
    # Generate predictions
    predict_input_fn = tf.estimator.inputs.numpy_input_fn({"tweetText": np.array(raw_test).astype(np.str)}, shuffle=False)
    results = estimator.predict(predict_input_fn)
    mCnt = 0
    for tweetEmotions in results:
        print(raw_test[mCnt].encode(encoding='UTF-8'))
        top_2 = tweetEmotions['probabilities'].argsort()[-2:][::-1]
        for emot in top_2:
            text_emot = encoder.classes_[emot]
            print(text_emot + ': ' + str(round(tweetEmotions['probabilities'][emot] * 100, 2)) + '%')
        print('')
        mCnt = mCnt + 1

if __name__ == "__main__":
    main()

KeyError: ignored